In [6]:
import numpy as np 

traj=np.load("test_site/trajectories/test_001BC50C70027E08.npy",allow_pickle=True)


print(traj)




[[[-148.78636832  -48.88713518    0.        ]
  [-253.98947848  -90.09586532    1.        ]
  [-275.96933096  -94.10959927    2.        ]
  [-304.91210719 -107.99226383    3.        ]
  [-348.54648376 -155.08971043    4.        ]
  [-392.13865105 -175.67937884    5.        ]
  [-424.89771505 -175.24561815    6.        ]
  [-436.72268681 -180.20823865    7.        ]
  [-434.46477052 -180.77629542    8.        ]
  [-422.37791278 -170.8612519     9.        ]
  [-396.42666858 -109.54001865   10.        ]
  [-371.42074125  -24.61590021   11.        ]
  [-359.56048024   15.34667734   12.        ]
  [-350.01452479   31.61439401   13.        ]
  [-345.51034424   54.43742519   14.        ]
  [-341.26198256   69.98792138   15.        ]
  [-336.06803135   78.68213895   16.        ]]]
